## Q. [마케팅] 자동차 시장 세분화
- 자동차 회사는 새로운 전략을 수립하기 위해 4개의 시장으로 세분화했습니다.
- 기존 고객 분류 자료를 바탕으로 신규 고객이 어떤 분류에 속할지 예측해주세요!


- 예측할 값(y): "Segmentation" (1,2,3,4)
- 평가: Macro f1-score
- data: train.csv, test.csv
- 제출 형식: 
~~~
ID,Segmentation
458989,1
458994,2
459000,3
459003,4
~~~

### 답안 제출 참고
- 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용 
- pd.DataFrame({'ID': test.ID, 'Segmentation': pred}).to_csv('003000000.csv', index=False)

### 노트북 구분
- basic: 수치형 데이터만 활용 -> 학습 및 test데이터 예측
- intermediate: 범주형 데이터도 활용 -> 학습 및 test데이터 예측
- advanced: 학습 및 교차 검증(모델 평가) -> 하이퍼파라미터 튜닝 -> test데이터 예측

### 학습을 위한 채점
- 최종 파일을 "수험번호.csv"가 아닌 "submission.csv" 작성 후 오른쪽 메뉴 아래 "submit" 버튼 클릭 -> 리더보드에 점수 및 등수 확인 가능함
- pd.DataFrame({'ID': test.ID, 'Segmentation': pred}).to_csv('submission.csv', index=False)


## 🧑‍💻 내 풀이

In [1]:
# 데이터 기초 정보 확인
import pandas as pd
x_train = pd.read_csv('/kaggle/input/big-data-analytics-certification-kr-2022/train.csv')
# y_train = x_train.pop('Segmentation')
# x_train
# y_train
x_test = pd.read_csv('/kaggle/input/big-data-analytics-certification-kr-2022/test.csv')
# x_test

# print(x_train.info())
# 결측치가 없군

x_train_ID = x_train.pop('ID')
x_test_ID = x_test.pop('ID')

# LabelEncoding하기
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

x_train['Gender'] = encoder.fit_transform(x_train['Gender'])
x_train['Ever_Married'] = encoder.fit_transform(x_train['Ever_Married'])
x_train['Graduated'] = encoder.fit_transform(x_train['Graduated'])
x_train['Profession'] = encoder.fit_transform(x_train['Profession'])
x_train['Spending_Score'] = encoder.fit_transform(x_train['Spending_Score'])
x_train['Var_1'] = encoder.fit_transform(x_train['Var_1'])

x_test['Gender'] = encoder.fit_transform(x_test['Gender'])
x_test['Ever_Married'] = encoder.fit_transform(x_test['Ever_Married'])
x_test['Graduated'] = encoder.fit_transform(x_test['Graduated'])
x_test['Profession'] = encoder.fit_transform(x_test['Profession'])
x_test['Spending_Score'] = encoder.fit_transform(x_test['Spending_Score'])
x_test['Var_1'] = encoder.fit_transform(x_test['Var_1'])


# x_train.info()
# x_train.corr()

# Ever_Married와 Spending_Score가 상관관계가 높으므로(0.6 이상) Spending_Score 삭제
# x_train.drop(columns = ['Spending_Score', 'Age'], inplace = True)
# x_test.drop(columns = ['Spending_Score', 'Age'], inplace = True)

# y_train 만들기
y_train = x_train.pop('Segmentation')
# print(y_train)

# x_train, x_test 스케일링하기
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()

from sklearn.preprocessing import RobustScaler
scaler2 = RobustScaler()

# 표준화
x_train = pd.DataFrame(scaler1.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler1.fit_transform(x_test), columns = x_test.columns)

# # 로버스트 스케일링
# x_train = pd.DataFrame(scaler2.fit_transform(x_train), columns = x_train.columns)
# x_test = pd.DataFrame(scaler2.fit_transform(x_test), columns = x_test.columns)

# 지금 제대로 알고 있는 것이 의사결정나무밖에 없으므로 그거라도 써보기
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth = 5, criterion = 'entropy')

# 학습
model.fit(x_train, y_train)

# 예측
y_test = pd.DataFrame(model.predict(x_test))
y_test

result = pd.concat([x_test_ID, y_test], axis = 1)
result.columns = ['ID', 'Segmentation']
result.to_csv('Submission.csv', index = False)

# score : 0.32147





# # XGBoost 분류기 써보기
# from xgboost import XGBRegressor
# model = XGBRegressor(n_estimatios = 100, max_depth = 3)
# model.fit(x_train, y_train)

# # 예측
# y_test = pd.DataFrame(model.predict(x_test))
# # y_test

# result = pd.concat([x_test_ID, y_test], axis = 1)
# result.columns = ['ID', 'Segmentation']
# result['Segmentation'] = round(result['Segmentation'], 0).astype(int)

# result.to_csv('Submission.csv', index = False)
# result